In [30]:
#imports
import pandas as pd
import numpy as np
import math


ModuleNotFoundError: No module named 'matplotlib'

In [2]:
#data imports
regions = pd.read_csv(r'risk_data.csv', sep=";", usecols=[0])
risk_data = pd.read_csv(r'risk_data.csv', sep=";", index_col=0)
asset_values = pd.read_csv(r'asset_values.csv', sep = ";", index_col=0)
influences = pd.read_csv(r'influences.csv', sep=";", index_col = 0)
base_data = pd.read_csv(r'base_data.csv', sep=";", index_col=0)

In [3]:
agg_risk = risk_data.sum(axis=1)

In [16]:
#risk per capita
rpk = agg_risk/base_data["pop"]
yearly_wage = base_data["sim_wage"]*52


In [18]:
#transfer payment method 1
transfer_payment = 0.2*sum(yearly_wage)*(rpk/sum(rpk))-0.2*yearly_wage


In [28]:
#transfer paymenet method 2
# transferring according to risk and population density, rationale: its cheeaper to protect a city than several farms

#normalizing risk and population density
risk_pc_norm = rpk/sum(rpk)
pop_density = base_data["pop"]/base_data["area"]
pop_density_norm = pop_density/sum(pop_density)

#creating a new combined value from the normalized pop_density and risk
combined = risk_pc_norm + pop_density_norm
combined_norm = combined/sum(combined)

#### transfer method Robin Hood: take percentage and distribute
transfer_payment_2 = -0.2*yearly_wage + 0.2*sum(yearly_wage)*(combined_norm)

#initializing base states - I think not really necessary since we used it to calculate b in the Excel. Only for welfare calc
v_base = base_data["b_i"]*yearly_wage/rpk
PI_base = v_base/sum(v_base)
H_base = PI_base*sum(base_data["pop"])
A_base = np.empty(len(agg_risk), dtype="float")
for i in range(len(agg_risk)):
    A_base[i] = math.pow(H_base[i], (base_data["prod"][i]/100))

#time_step_0
v_0=(base_data["b_i"]*(yearly_wage+transfer_payment_2)/rpk)
PI_0=v_0/sum(v_0)
H_0=PI_0*sum(base_data["pop"])
pop_dens_0=H_0/base_data["area"]
#wage level adaption based on population density --> Linear regression on weekly wage shows beta_0 = 598.74 and beta_1 = 0.0359
wage_1 = (0.0359*pop_dens_0+598.74)*52

#time_step_1
rpk_1 = agg_risk/H_0
v_1=(base_data["b_i"]*(wage_1+transfer_payment_2)/rpk_1)
PI_1 = v_1/sum(v_1)
H_1 = PI_1*sum(base_data["pop"])
A_1 = np.empty(len(agg_risk), dtype="float")
for i in range(len(agg_risk)):
    A_1[i] = math.pow(H_1[i], (base_data["prod"][i]/100)) #assumption: Productivity is based on regional factors and does not change

welfare_increase = sum(A_1)-sum(A_base)
inc_perc = 100*welfare_increase/sum(A_base)

print(welfare_increase)


### transfer method 2: try every percentage of taxes and compare resulting welfare
welfare_increase_list = []
percentage = []
for i in range(101):
    transfer_payment_2 = -(i/100)*yearly_wage + + (i/100)*sum(yearly_wage)*(combined_norm)
    percentage.append(i/100)

    #sorry for the mess, just the same as before
    #initializing base states - I think not really necessary since we used it to calculate b in the Excel. Only for welfare calc
    v_base = base_data["b_i"]*yearly_wage/rpk
    PI_base = v_base/sum(v_base)
    H_base = PI_base*sum(base_data["pop"])
    A_base = np.empty(len(agg_risk), dtype="float")
    for i in range(len(agg_risk)):
        A_base[i] = math.pow(H_base[i], (base_data["prod"][i]/100))

    #time_step_0
    v_0=(base_data["b_i"]*(yearly_wage+transfer_payment_2)/rpk)
    PI_0=v_0/sum(v_0)
    H_0=PI_0*sum(base_data["pop"])
    pop_dens_0=H_0/base_data["area"]
    #wage level adaption based on population density --> Linear regression on weekly wage shows beta_0 = 598.74 and beta_1 = 0.0359
    wage_1 = (0.0359*pop_dens_0+598.74)*52

    #time_step_1
    rpk_1 = agg_risk/H_0
    v_1=(base_data["b_i"]*(wage_1+transfer_payment_2)/rpk_1)
    PI_1 = v_1/sum(v_1)
    H_1 = PI_1*sum(base_data["pop"])
    A_1 = np.empty(len(agg_risk), dtype="float")
    for i in range(len(agg_risk)):
        A_1[i] = math.pow(H_1[i], (base_data["prod"][i]/100)) #assumption: Productivity is based on regional factors and does not change

    welfare_increase = sum(A_1)-sum(A_base)
    inc_perc = 100*welfare_increase/sum(A_base)
    welfare_increase_list.append(welfare_increase)

print(welfare_increase_list)


84025261993.09097
[-29706.109603881836, 3136651143.7872696, 6394050506.556343, 9771560259.521416, 13268356759.415771, 16883405320.571121, 20615465378.861115, 24463096301.302895, 28424663798.38566, 32498346893.774345, 36682145404.03483, 40973887879.45037, 45371239955.84518, 49871713066.59167, 54472673463.64301, 59171351496.48931, 63964851098.362404, 68850159429.79282, 73824156630.72379, 78883625633.79648, 84025261993.09097, 89245683684.53043, 94541440836.28815, 99909025349.85059, 105344880374.85812, 110845409603.43576, 116406986352.40677, 122025962404.53247, 127698676582.70547, 133421463033.82242, 139190659201.8494, 145002613472.34503, 150853692473.41116, 156740288020.66342, 162658823696.3599, 168605761055.2692, 174577605452.1792, 180570911488.16302, 186582288074.78397, 192608403117.36548, 198645987820.2457, 204691840618.59396, 210742830742.86496, 216795901423.34625, 222848072743.46088, 228896444151.57648, 234938196642.01352, 240970594616.75858, 246990987440.06985, 252996810698.73233, 2

In [24]:
#initializing base states - I think not really necessary since we used it to calculate b in the Excel. Only for welfare calc
v_base = base_data["b_i"]*yearly_wage/rpk
PI_base = v_base/sum(v_base)
H_base = PI_base*sum(base_data["pop"])
A_base = np.empty(len(agg_risk), dtype="float")
for i in range(len(agg_risk)):
    A_base[i] = math.pow(H_base[i], (base_data["prod"][i]/100))

In [25]:
#time_step_0
v_0=(base_data["b_i"]*(yearly_wage+transfer_payment)/rpk)
PI_0=v_0/sum(v_0)
H_0=PI_0*sum(base_data["pop"])
pop_dens_0=H_0/base_data["area"]
#wage level adaption based on population density --> Linear regression on weekly wage shows beta_0 = 598.74 and beta_1 = 0.0359
wage_1 = (0.0359*pop_dens_0+598.74)*52

#time_step_1
rpk_1 = agg_risk/H_0
v_1=(base_data["b_i"]*(wage_1+transfer_payment)/rpk_1)
PI_1 = v_1/sum(v_1)
H_1 = PI_1*sum(base_data["pop"])
A_1 = np.empty(len(agg_risk), dtype="float")
for i in range(len(agg_risk)):
    A_1[i] = math.pow(H_1[i], (base_data["prod"][i]/100)) #assumption: Productivity is based on regional factors and does not change


In [27]:
welfare_increase = sum(A_1)-sum(A_base)
inc_perc = 100*welfare_increase/sum(A_base)
print(welfare_increase)

-17017605825.303848


In [116]:
print("The increase in welfare with the given transfer policy is %s which is an increase of %s percent." % (welfare_increase, inc_perc))

The increase in welfare with the given transfer policy is -17017605825.303848 which is an increase of -37.88639120198049 percent.
